## Data Story
### Instacart Market Basket Analysis EDA: Feature Set
----
Ryan Alexander Alberts

6/27/2017

#### In this notebook, I want to explore features that may drive a customer's decision strategy for their next order.

----

* __Define bounds of test set:__
    * __Products__ (for each customer)
        * all previously ordered products
        * categories for all products
        * total number of products
        * variability of product count across orders
        * [reorder > 1] --vs-- [reorder == 1 and in most recent order]
    * __Customers__
        * reorder rate (% of products w/ 'reordered' == 1)
        * order count
        * for last order|product: # of orders since last occurance
    * __Basket Size__
        * max, min, avg. product count per customer
        * weighted avg. product count (more recent weighted more heavily)
    * __'None'__
        * subset of customers|orders for which 'None' may work
        * behavior in context of 'ideal' order product count


* __Explore baseline submissions__
    * orders containing all previously ordered products
    * orders including subsets of previously ordered products 
        * based on frequency
        * based on most recent order(s)


* __Future Topics__
    * Order Frequency
    * Days since last order
    * day of the week, hour of the day
    * Macro-level trends in time-data, like spikes in product count in first xx% or last xx% of all customers orders, corresonding to Summer or holidays
    * Weekday vs. Weekend
    * 


---


In [1]:
import pandas as pd
import re
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set_style("whitegrid")
import calendar

In [2]:
# First, let's import requisite files
orders      = pd.read_csv('../Instacart_Input/orders.csv')
prior_set   = pd.read_csv('../Instacart_Input/order_products__prior.csv')
train_set   = pd.read_csv('../Instacart_Input/order_products__train.csv')
aisles      = pd.read_csv('../Instacart_Input/aisles.csv')
departments = pd.read_csv('../Instacart_Input/departments.csv')
products    = pd.read_csv('../Instacart_Input/products.csv')

In [3]:
orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


In [4]:
test = orders[orders['eval_set'] == 'test']
test.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
38,2774568,3,test,13,5,15,11.0
44,329954,4,test,6,3,12,30.0
53,1528013,6,test,4,3,16,22.0
96,1376945,11,test,8,6,11,8.0
102,1356845,12,test,6,1,20,30.0


In [5]:
# order history for all user_ids in test set
test_user_orders = orders[orders['user_id'].isin(test['user_id'].values)]
test_user_orders = test_user_orders.merge(prior_set, on='order_id')
test_user_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,1374495,3,prior,1,1,14,NaN,9387,1,0
1,1374495,3,prior,1,1,14,NaN,17668,2,0
2,1374495,3,prior,1,1,14,NaN,15143,3,0
3,1374495,3,prior,1,1,14,NaN,16797,4,0
4,1374495,3,prior,1,1,14,NaN,39190,5,0


In [6]:
# Let's add in department and aisle data, now that we have products

products_df = pd.merge(left=pd.merge(products, right=departments, on='department_id'), 
                       right=aisles, on='aisle_id')
products_df = products_df.drop(['department_id', 'aisle_id'], axis=1)
test_user_orders = test_user_orders.merge(products_df, on='product_id')
test_user_orders.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,department,aisle
0,1374495,3,prior,1,1,14,NaN,9387,1,0,Granny Smith Apples,produce,fresh fruits
1,2037211,3,prior,4,2,18,20.0,9387,3,1,Granny Smith Apples,produce,fresh fruits
2,2710558,3,prior,5,0,17,12.0,9387,2,1,Granny Smith Apples,produce,fresh fruits
3,1972919,3,prior,6,0,16,7.0,9387,3,1,Granny Smith Apples,produce,fresh fruits
4,1839752,3,prior,7,0,15,7.0,9387,9,1,Granny Smith Apples,produce,fresh fruits


In [7]:
# For each customer, create an array of order_ids, 
# and for each order_id, an array of products

user_orderlist = pd.DataFrame(test_user_orders.groupby('user_id')
                              ['order_id'].apply(list))
order_productlist = pd.DataFrame(test_user_orders.groupby('order_id')
                                 ['product_id'].apply(list))
user_orderlist.head()

,order_id
user_id,
3,"[1374495, 2037211, 2710558, 1972919, 1839752, ..."
4,"[3343014, 3343014, 3343014, 691089, 3343014, 2..."
6,"[2086598, 298250, 2086598, 2086598, 298250, 20..."
11,"[2411567, 2921164, 2411567, 2411567, 2840752, ..."
12,"[221248, 221248, 1916106, 1247527, 1057378, 22..."


In [8]:
# prettify dataframes
user_orderlist.reset_index(level='user_id', inplace=True)
user_orderlist.columns = ['user_id', 'orders_array']

order_productlist.reset_index(level='order_id', inplace=True)
order_productlist.columns = ['order_id', 'products_array']
user_orderlist.head()

,user_id,orders_array
0,3,"[1374495, 2037211, 2710558, 1972919, 1839752, ..."
1,4,"[3343014, 3343014, 3343014, 691089, 3343014, 2..."
2,6,"[2086598, 298250, 2086598, 2086598, 298250, 20..."
3,11,"[2411567, 2921164, 2411567, 2411567, 2840752, ..."
4,12,"[221248, 221248, 1916106, 1247527, 1057378, 22..."


In [9]:
# Let's add departments and aisles now that we have order product data

order_categorylist = pd.DataFrame(test_user_orders.groupby('order_id')
                              ['department'].apply(list).apply(np.unique))
order_sub_categorylist = pd.DataFrame(test_user_orders.groupby('order_id')
                              ['aisle'].apply(list).apply(np.unique))

In [10]:
# Prettify DataFrames
order_categorylist.reset_index(level='order_id', inplace=True)
order_categorylist.columns = ['order_id', 'departments_array']

order_sub_categorylist.reset_index(level='order_id', inplace=True)
order_sub_categorylist.columns = ['order_id', 'aisles_array']
order_categorylist.head()

,order_id,departments_array
0,13,"[alcohol, beverages, dairy eggs, pantry, snacks]"
1,15,"[beverages, canned goods, pantry, personal care]"
2,16,"[beverages, snacks]"
3,18,"[babies, beverages, bulk, canned goods, dairy ..."
4,24,[frozen]


In [56]:
order_reorderlist = pd.DataFrame(test_user_orders.groupby('order_id')
                              ['reordered'].apply(list))
# Prettify DataFrames
order_reorderlist.reset_index(level='order_id', inplace=True)
order_reorderlist.columns = ['order_id', 'reordered_array']

order_reorderlist.head()

,order_id,reordered_array
0,13,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,15,"[1, 1, 1, 1, 1]"
2,16,"[0, 0, 1]"
3,18,"[1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, ..."
4,24,"[0, 0, 0]"


In [73]:
# Let's add reorder rates and product counts for each order

order_reorderlist['reorder_rate'] = 0.0
order_reorderlist['product_count'] = 0

for i in range(len(order_reorderlist)):
    reorderlist = order_reorderlist['reordered_array'][i]
    order_reorderlist.set_value(i, 'product_count', len(reorderlist))
    if 1 in reorderlist:
        order_reorderlist.set_value(i, 'reorder_rate', round(np.sum(reorderlist) / float(len(reorderlist))*100, 2))
    
order_reorderlist.tail()

,order_id,reordered_array,reorder_rate,product_count
1167492,3421076,"[1, 1, 1, 1, 1, 1, 1, 0]",87.50,8
1167493,3421077,"[0, 0, 0, 0]",0.00,4
1167494,3421078,"[1, 1, 0, 1, 1, 0, 1, 1, 1]",77.78,9
1167495,3421079,[0],0.00,1
1167496,3421083,"[1, 1, 0, 1, 1, 0, 0, 0, 0, 0]",40.00,10


In [ ]:
# We have:
#  1. Slightly overwhelming DataFrame with complete order history for test users
#  2. More approachable DataFrames with orders         for each test user
#  3.                                   products       for each test user's orders
#  4.                                   product counts for each test user's orders
#  5.                                   departments    for each test user's orders
#  6.                                   aisles         for each test user's orders
#  7.                                   reorder rates  for each test user's orders
